# Auto post to Social site
Schedule: 1 Image post per day per Page (setup n cron items, each item has 1 category in param)

In [119]:
from PIL import Image, ImageDraw, ImageFont
import pymongo
import os
import requests
import time

In [120]:
from dotenv import load_dotenv
load_dotenv(override=True) 
LI_URI = os.environ['LI_URI']
LI_REST_URI = os.environ['LI_REST_URI']
LI_VERSION = os.environ['LI_VERSION']

In [121]:
db_client = pymongo.MongoClient(os.environ['REMOTE_MONGO_DB'])
db = db_client['db_certificates']   
tb_advertising_posts = db['tb_advertising_posts']   #store contents to post in social media
tb_cert_metadata = db['tb_cert_metadata']    #meta data of certificates
tb_linkedin_app = db['tb_linkedin_app']    #LI App
tb_linkedin_page = db['tb_linkedin_page']    #LI Page info

In [122]:
def get_timestamp():
    return str(int(time.time()))

def wrap_text(text, font, max_width, draw):
    """
    Splits text into lines that fit within max_width
    """
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        test_line = current_line + (" " if current_line else "") + word
        w = draw.textlength(test_line, font=font)

        if w <= max_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    return lines

def draw_wrapped_text(draw, position, text, font, max_width, fill=(255,255,255), line_spacing=6):
    x, y = position
    lines = wrap_text(text, font, max_width, draw)

    for line in lines:
        draw.text((x, y), line, font=font, fill=fill)
        y += font.size + line_spacing


In [123]:
#current path of this project
CURRENT_PATH = '/Users/sangdo/Documents/Source/Python/python_webscrap/cert_exam/'

In [124]:
CTA_LINKS = '\nVisit the links above to help you pass these certifications at your first try\n'
CTA_LINKS_COMMENT = '\nCheck out the practice questions in the comments to help you pass these certifications on your first try\n'

In [125]:
#define 5 carousel templates
CAROUSEL_TEMPLATE_POSITION_MAPPING = [
    #carousel 1
    {
        'folder_name': 'carousel_1',
        'header_font_size': 50,
        'content_font_size': 40,
        'position_mapping': [
            #slide 1: hook
            {
                "header": (60, 285)
            },
            #slide 2: problems
            {
                "header": (115, 390),
                "content": (115, 680)
            },
            #slide 3: insight
            {
                "header": (115, 390),
                "content": (115, 680)
            },
            #slide 4: values
            {
                "header": (115, 390),
                "content": (115, 680)
            },
            #slide 5: logo
            {
                "link": (53, 100),
                "link_w": 980,
                "logo": [(53, 290), (53, 630), (53, 975)], #list of positions of each logo
                "title": [(370, 290), (370, 630), (370, 975)], #list of positions of each logo
                "title_w": 700,
                "salary": [(370, 535), (370, 880), (370, 1210)] #list of positions of each logo
            }
        ]
    }
]

In [139]:
IMAGE_TEMPLATE_MAPPING = {
    'header_font_size': 50,     #for salary and header
    'content_font_size': 40,
    'max_text_w': 1100,
    'position_mapping': {
        'salary': (310, 55),
        'hook': (60, 155),
        'sub_content_l': 60,
        'sub_content_t': 390,
        # 'sub_content_2': (60, 515),
        # 'sub_content_3': (60, 630),
        'logos': [(60, 740), (450, 740), (835, 740)]
    }
}

In [127]:
#2 or 3 certifications in 1 image
LOGO_POSITIONS = [(70, 290), (70, 630), (70, 985)]
NAME_POSITIONS = [(410, 350), (410, 695), (410, 1035)]  #certs name
SALARY_POSITION = (200, 130)    #salary range
FOOTER_URL_POSITION = (330, 1305)   #https://udemy/...

SINGLE_IMG_TEMPLATE_PATH = CURRENT_PATH + 'template/imgs/'

IMG_TEMPLATE_PATH = CURRENT_PATH + 'template/img_template.png'
IMG_LOGO_PATH_PREFIX = CURRENT_PATH + 'logo/'

IMG_OUTPUT_PATH_PREFIX = CURRENT_PATH + 'output/'  #the final image
FONT_PATH = CURRENT_PATH + "font/Swansea-q3pd.ttf"

In [128]:
#test data
IMG_CONTENT = {
        'symbol': ['NCP_AAI', 'SNOW_SSGA', 'ISACA_AAISM'],
        'hook': 'Architect roles aren\'t about knowing more tools. They\'re about owning bigger decisions.',
        'sub_contents':[
            'Qualify for architect-level roles with higher salary ceilings',
            'Gain leverage in compensation, title, and scope discussions',
            'Reduce competition by signaling readiness for complex, cross-domain systems'
        ],
        'salary': '$120k - $190k'
}

SINGLE_TEMPLATE_IMG_FILENAME = 'template_1.png'

In [137]:
#output: 1 image with multiple certifications from single image template
def create_certifications_from_single_image_template(
    post_info
):
    base = Image.open(SINGLE_IMG_TEMPLATE_PATH + SINGLE_TEMPLATE_IMG_FILENAME).convert("RGBA")
    draw = ImageDraw.Draw(base)
    #draw salary range
    draw_wrapped_text(
        draw=draw,
        position= IMAGE_TEMPLATE_MAPPING['position_mapping']['salary'],
        text= 'Salary: ' + post_info['salary'],
        font= ImageFont.truetype(FONT_PATH, 60),
        max_width= 590,
        fill= (255, 255, 255)   #text color
    )
    #draw hook
    draw_wrapped_text(
        draw=draw,
        position= IMAGE_TEMPLATE_MAPPING['position_mapping']['hook'],
        text= post_info['hook'],
        font= ImageFont.truetype(FONT_PATH, 60),
        max_width= IMAGE_TEMPLATE_MAPPING['max_text_w'],
        fill= (0, 0, 0)   #text color
    )
    #draw sub contents
    index = 0
    for sub_content in post_info['sub_contents']:
        draw_wrapped_text(
            draw=draw,
            position= (IMAGE_TEMPLATE_MAPPING['position_mapping']['sub_content_l'], IMAGE_TEMPLATE_MAPPING['position_mapping']['sub_content_t'] + index * 125),
            text= sub_content,
            font= ImageFont.truetype(FONT_PATH, 40),
            max_width= IMAGE_TEMPLATE_MAPPING['max_text_w'],
            fill= (0, 0, 0)   #text color
        )
        index = index + 1
    #draw logos
    index = 0
    for symbol in post_info['symbols']:
        overlay_img = Image.open(IMG_LOGO_PATH_PREFIX + symbol + '.png').convert("RGBA")
        base.paste(overlay_img, IMAGE_TEMPLATE_MAPPING['position_mapping']['logos'][index], overlay_img)
        index = index + 1

    base.convert("RGB").save(IMG_OUTPUT_PATH_PREFIX + '20261215.png', "PNG")
    print(f"Final image saved to: {IMG_OUTPUT_PATH_PREFIX}")
    #print link for each cert
    for symbol in post_info['symbols']:
        cert_details = tb_cert_metadata.find_one({'symbol':symbol})
        print(cert_details['name'])
        print('https://certification-pro.blogspot.com/2026/02/' + cert_details['slug'] + '.html')


#test
# create_certifications_from_single_image_template()

In [141]:
#find a daily post
def export_daily_post(_category):
    tb_advertising_posts = db['tb_advertising_posts']
    post = tb_advertising_posts.find_one({'posted': 0, 'category': _category})
    if post is not None:
        create_certifications_from_single_image_template(post)
#test
export_daily_post('AI')

Final image saved to: /Users/sangdo/Documents/Source/Python/python_webscrap/cert_exam/output/
PMI Cognitive Project Management in AI (CPMAI)
https://certification-pro.blogspot.com/2026/02/pmi-cognitive-project-management-in-ai-cpmai.html
AWS Certified AI Practitioner (AIF-C01)
https://certification-pro.blogspot.com/2026/02/aws-certified-ai-practitioner-aif-c01.html
Microsoft Certified: Azure AI Fundamentals (AZ-AI-900)
https://certification-pro.blogspot.com/2026/02/microsoft-certified-azure-ai-fundamentals-az-ai-900.html


In [ ]:
#output: 1 image with multiple certifications in slide
def create_certifications_image(
    page_link:str,
    cert_symbols: list,
    cert_names: list,
    salary_range: str,
    output_path: str
):
    base = Image.open(IMG_TEMPLATE_PATH).convert("RGBA")
    draw = ImageDraw.Draw(base)
    index = 0
    # ---- Add logo images ----
    for symbol in cert_symbols:
        overlay_img = Image.open(IMG_LOGO_PATH_PREFIX + symbol + '.png').convert("RGBA")
        #don't need to resize because logos are saved with size 330x330
        # if "size" in item and item["size"]:
        #     overlay_img = overlay_img.resize(item["size"], Image.ANTIALIAS)
        base.paste(overlay_img, LOGO_POSITIONS[index], overlay_img)
        index = index + 1

    # ---- Add certification names ----
    index = 0
    for t in cert_names:
        font = ImageFont.truetype(FONT_PATH, 48)
        draw_wrapped_text(
            draw=draw,
            position= NAME_POSITIONS[index],
            text= t,
            font= font,
            max_width= 620,
            fill= (0, 0, 153),   #certification name color
            line_spacing=8
        )
        index = index + 1
    #draw salary range
    draw_wrapped_text(
            draw=draw,
            position= SALARY_POSITION,
            text= 'Salary range: ' + salary_range,
            font= ImageFont.truetype(FONT_PATH, 60),
            max_width= 800,
            fill= (153, 153, 0)   #text color
        )
    #draw footer
    draw_wrapped_text(
            draw=draw,
            position= FOOTER_URL_POSITION,
            text= page_link,
            font= ImageFont.truetype(FONT_PATH, 20),
            max_width= 620,
            fill= (0, 0, 255)
        )

    base.convert("RGB").save(output_path, "PNG")
    print(f"Final image saved to: {output_path}")


In [ ]:
#share new post with 1 new uploaded image
def share_my_img_2_LI(asset, owner_id, access_token, post_content):
    #print('asset: ' + asset)
    payload = {
        "author": owner_id,
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {
                    "text": post_content
                },
                "shareMediaCategory": "IMAGE",
                "media": [
                    {
                        "status": "READY",
                        "media": asset
                    }
                ]
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
        }
    }
    li_headers = {
        'Authorization': access_token,
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    try:
        response = requests.post(LI_URI + 'v2/ugcPosts', json=payload, headers=li_headers)
        # print('====== Share Post to Page results:')
        # print("Status Code:", response.status_code)
        # print("Headers:", response.headers)
        # print("Response Body:", response.text)
        
        if response.status_code >= 200 and response.status_code < 300:
            print("The image was shared successfully!")
            return 'ok'
        else:
            print("The image was shared failed.")
            return 'failed'
    except Exception as e:
        print(e)   
        return 'failed'

In [ ]:
#upload image to LI
def upload_img_2_LI_and_share(li_access_token, file_path, upload_detail, owner_id, post_content):
    uploadUrl = upload_detail['uploadMechanism']['com.linkedin.digitalmedia.uploading.MediaUploadHttpRequest']['uploadUrl']
    #upload the image https://learn.microsoft.com/en-us/linkedin/consumer/integrations/self-serve/share-on-linkedin#upload-image-or-video-binary-file
    try:
        with open(file_path, 'rb') as file:
            files = {'file': (file.name, file, 'image/jpg')}
            headers = {
                'Authorization': li_access_token,
                'LinkedIn-Version': LI_VERSION
            }

            response = requests.post(uploadUrl, files=files, headers=headers)
            # print('====== Upload image results:')
            # print("Status Code:", response.status_code)
            # print("Headers:", response.headers)
            # print("Response Body:", response.text)

            if response.status_code >= 200 and response.status_code < 300:
                print("File uploaded successfully!")
                time.sleep(5)   #delay 5 seconds for image going through LI system
                result = share_my_img_2_LI(upload_detail['asset'], owner_id, li_access_token, post_content)
                return result
            else:
                print("File upload failed.")
                return 'failed'

    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return 'failed' 
    except requests.exceptions.RequestException as e:
        print(f"Error during upload: {e}")
        return 'failed'
    

In [ ]:
#get upload link from LI
def get_LI_upload_link(owner_id, li_headers):
    url = LI_URI + 'v2/assets?action=registerUpload'
    payload = {
        "registerUploadRequest": {
            "recipes": [
                "urn:li:digitalmediaRecipe:feedshare-image"
            ],
            "owner": owner_id,
            "serviceRelationships": [
                {
                    "relationshipType": "OWNER",
                    "identifier": "urn:li:userGeneratedContent"
                }
            ]
        }
    }
    try:
        detail = requests.post(url, json=payload, headers=li_headers)
        return detail.json()
    except Exception as e:
        print(e)
        return {'error': e}

In [ ]:
def post_1_image_by_category(_category):
    post_details = find_1_content_from_db(_category)
    if post_details is None:
        print('There is no post for Image content of category: ' + _category)
        return
    if 'symbols' not in post_details:
        print('There is no symbol for Image content of category: ' + _category)
        return
    symbols = post_details['symbols'].split(',')
    #find name of certifications
    cert_names = []
    cert_links = []
    for symbol in symbols:
        cert_details = tb_cert_metadata.find_one({'symbol':symbol})
        if cert_details is None:
            print('There is no detail for the certification: ' + symbol)
            return
        cert_names.append(cert_details['name'])
        cert_links.append(cert_details['udemy_link'])
     #find app token and page linking to this category
    page_info = tb_linkedin_page.find_one({'category': _category})
    if page_info is None:
        print('There is no page info for category: ' + _category)
        return
    #create final image
    new_img_path = IMG_OUTPUT_PATH_PREFIX + post_details['symbols'].replace(',', '-') + '.png'
    create_certifications_image(page_info['link'], symbols, cert_names, post_details['salary'], new_img_path)
    #find app info
    app_info = tb_linkedin_app.find_one({'app_id': page_info['app_id']})
    if app_info is None:
        print('There is no app info for category: ' + _category)
        return
    #create CTA link
    cta_links = []
    index = 1
    for cert_name in cert_names:
        cta_links.append(str(index) + '. ' + cert_name + ': ' + cert_links[index-1] + '\n')
        index = index + 1
    cta_links_text = CTA_LINKS + ''.join(cta_links)
    post_content = post_details['content'] + cta_links_text
    #upload image to to Linkedin Page
    li_access_token = 'Bearer ' + app_info['access_token']
    li_headers = {
        'Authorization': li_access_token,
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    owner_id = "urn:li:organization:" + page_info['page_id']    #my page
    upload_link = get_LI_upload_link(owner_id, li_headers)
    result_upload = upload_img_2_LI_and_share(li_access_token, new_img_path, upload_link['value'], owner_id, post_content)
    if result_upload == 'ok':
        #the post is successfully up
        #delete that image
        if os.path.exists(new_img_path):
            os.remove(new_img_path)
        print('==== The image is shared in LI Page for category: ' + _category)
        #add flag to that post to indicate done
        post_details['posted_li_img'] = 1
        tb_advertising_posts.replace_one({"_id": post_details['_id']}, post_details)
        return
    else:
        print('There is an error for posting in LI page for category: ' + _category)
        return
#test
# post_1_image_by_category('CLOUD')

In [ ]:
# if __name__ == '__main__':
#     args = sys.argv
#     print(args)
#     if len(args) > 1:
#         category = args[1]   #category of certifications
#         print(category)
#         post_1_image_by_category(category)

In [ ]:
# db_client.close()